This file has the code to obtain syllable transition values from Onset-Offset data, and create a graph based on it

Date: 20/9/2022

In [3]:
import math
import pandas
import os

In [4]:
os.chdir('D:\\4th Year\\Semester 7\\BI4313 Sem Project\\IN-comparative-analysis\\IN-comparative-analysis\\OnsetOffsetFiles')

In [6]:
file = open("M09.txt", 'r')

file.readlines()

['FileName\tSyll #\tSyll Label\t Syll Onset (ms)\t Syll Offset (ms)\t Syll Duration (sec)\t Mean Frequency (Hz)\t Entropy\tLog Amplitude (dB)\tPitch Goodness\tFrequencyModulation\tAmplitudeModulation\tEntropyVariance\n',
 '20150622_M09_paired_01.wav\t1\ti\t9561.84\t9579.75\t0.0179138\t2537.39\t-3.5513\t11.4333\t45.3959\t45.3802\t-0.0129652\t0.455702\tNaN\t\n',
 '20150622_M09_paired_01.wav\t2\ta\t9734.56\t9977.57\t0.243016\t3284.31\t-2.9849\t19.2989\t41.5009\t42.234\t-0.00630934\t0.0866028\tNaN\t\n',
 '20150622_M09_paired_01.wav\t3\tb\t10056.8\t10129.2\t0.0723583\t2742.15\t-4.25822\t11.7839\t37.0944\t8.50343\t-0.0063626\t0.340397\tNaN\t\n',
 '20150622_M09_paired_01.wav\t4\tc\t10145.6\t10195.6\t0.0500907\t4611.29\t-2.52174\t22.2206\t43.7937\t45.7616\t-0.06946\t0.147445\tNaN\t\n',
 '20150622_M09_paired_01.wav\t5\td\t10247.6\t10379.9\t0.132222\t4322.4\t-5.17878\t28.4537\t24.845\t11.7422\t-0.0117026\t0.576616\tNaN\t\n',
 '20150622_M09_paired_01.wav\t6\te\t10391.6\t10497.3\t0.105646\t6117.79